In [1]:
import sys
import logging
sys.path.append(r"C:\Users\Cedri\OneDrive\Documents\GitHub\2023-24d-fai2-adsai-group-cv5")

In [2]:
from src.utils.configuration import *
from src.features.feature_scaling import *
from src.data.data_preprocessing import DataPipelineSetup
from src.models.model_training import ModelTraining
from src.features.instance_segmentation import InstanceSegmentation
from src.features.root_coord_extraction import LandmarkExtraction

segmentation = InstanceSegmentation()
landmarks = LandmarkExtraction()
processor = DataPipelineSetup()
modelling = ModelTraining()

## STEP 1:
    creating folder in the repository
    unzipping the masks.zip and train.zip in images, root_masks, shoot_masks folders in data_unpatched
    note: when unzipping the folders masks and train will appear in notebook folder these folders can be deleted safely
     

In [3]:
print_folders()

FOLDER STRUCTURE HELPER
external_data_folder: D:\2023-24d-fai2-adsai-group-cv5\data\external
raw_data_folder: D:\2023-24d-fai2-adsai-group-cv5\data\raw
processed_data_folder: D:\2023-24d-fai2-adsai-group-cv5\data\processed
data_unpatched: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_unpatched
images_folder_unpatched: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_unpatched\images
test_folder: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_unpatched\test
root_folder_unpatched: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_unpatched\root_masks
shoot_folder_unpatched: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_unpatched\shoot_masks
data_patched: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_patched
images_folder_patched: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_patched\images\images
root_folder_patched: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_patched\root_masks\root_masks
shoot_folder_patched: D:\2023-24d-fai2-adsai-group-cv5\data\raw\data_patched\shoot_masks\shoot_ma


data/
│
├── processed/
│   ├── data_predicted/
│   └── data_predicted_clean/
│
└── raw/
    ├── data_patched/
    │   ├── image/
    │   │    └── images/
    │   ├── shoot_masks/
    │   │    └── shoot_masks/
    │   └── root_masks/
    │        └── root_masks/
    │
    └── data_unpatched/
        ├── image/
        ├── shoot_masks/
        ├── root_masks/
        └── test/

In [5]:
processor.create_folders()
processor.unzip("train")
processor.unzip("test")
processor.unzip("masks")

## STEP 2:
    cropping train set (images and masks) 
    if the images and masks are already the diserable size the cropping function won't modified any further

In [6]:
# NOTE the progress bar is made for the console and not jupyter
processor.crop(folder_config.get("images_folder_unpatched"))
processor.crop(folder_config.get("root_folder_unpatched"))
processor.crop(folder_config.get("shoot_folder_unpatched"))

100%|          | 0/88 [00:00<?, ?it/s]          | 1/88 [00:00<00:37,  2.32it/s]▏         | 2/88 [00:00<00:36,  2.33it/s]▎         | 3/88 [00:01<00:33,  2.52it/s]▍         | 4/88 [00:01<00:32,  2.60it/s]▌         | 5/88 [00:01<00:30,  2.69it/s]▋         | 6/88 [00:02<00:29,  2.75it/s]▊         | 7/88 [00:02<00:29,  2.76it/s]▉         | 8/88 [00:02<00:28,  2.80it/s]█         | 9/88 [00:03<00:28,  2.79it/s]█▏        | 10/88 [00:03<00:27,  2.86it/s]█▎        | 11/88 [00:04<00:27,  2.83it/s]█▎        | 12/88 [00:04<00:26,  2.85it/s]█▍        | 13/88 [00:04<00:26,  2.81it/s]█▌        | 14/88 [00:05<00:26,  2.83it/s]█▋        | 15/88 [00:05<00:26,  2.80it/s]█▊        | 16/88 [00:05<00:25,  2.82it/s]█▉        | 17/88 [00:06<00:25,  2.79it/s]██        | 18/88 [00:06<00:24,  2.84it/s]██▏       | 19/88 [00:06<00:24,  2.82it/s]██▎       | 20/88 [00:07<00:24,  2.83it/s]██▍       | 21/88 [00:07<00:23,  2.83it/s]██▌       | 22/88 [00:07<00:23,  2.84it/s]██▌       | 23/88 [00:08<00:23,  2.82it/s]██▋  

## STEP 3:
    creating patches for both masks and images and storing them in data_patched

In [9]:
# NOTE the progress bar is made for the console and not jupyter
_,_ = processor.img_patchify(folder_config.get("images_folder_unpatched"), folder_config.get("images_folder_patched"))
_,_ = processor.img_patchify(folder_config.get("root_folder_unpatched"), folder_config.get("root_folder_patched"))
_,_ = processor.img_patchify(folder_config.get("shoot_folder_unpatched"), folder_config.get("shoot_folder_patched"))

100%|          | 0/88 [00:00<?, ?it/s]          | 1/88 [00:00<00:35,  2.48it/s]▏         | 2/88 [00:00<00:34,  2.48it/s]▎         | 3/88 [00:01<00:33,  2.57it/s]▍         | 4/88 [00:01<00:31,  2.68it/s]▌         | 5/88 [00:01<00:30,  2.68it/s]▋         | 6/88 [00:02<00:30,  2.69it/s]▊         | 7/88 [00:02<00:29,  2.72it/s]▉         | 8/88 [00:02<00:29,  2.73it/s]█         | 9/88 [00:03<00:28,  2.73it/s]█▏        | 10/88 [00:03<00:28,  2.75it/s]█▎        | 11/88 [00:04<00:28,  2.73it/s]█▎        | 12/88 [00:04<00:27,  2.75it/s]█▍        | 13/88 [00:04<00:27,  2.71it/s]█▌        | 14/88 [00:05<00:27,  2.72it/s]█▋        | 15/88 [00:05<00:26,  2.73it/s]█▊        | 16/88 [00:05<00:26,  2.73it/s]█▉        | 17/88 [00:06<00:26,  2.72it/s]██        | 18/88 [00:06<00:25,  2.71it/s]██▏       | 19/88 [00:07<00:25,  2.69it/s]██▎       | 20/88 [00:07<00:25,  2.69it/s]██▍       | 21/88 [00:07<00:24,  2.70it/s]██▌       | 22/88 [00:08<00:24,  2.73it/s]██▌       | 23/88 [00:08<00:24,  2.69it/s]██▋  

## STEP 4:
    generating data for training and validation
    trains a model on predefined configurations (patch_size=255, epochs=50, validation_step=16, input_shape=(255,255,3), num_classes=1)

In [4]:
images_patched = os.path.dirname(folder_config.get("images_folder_patched"))
masks_patched = os.path.dirname(folder_config.get("root_folder_patched"))

_,_,_,_ = modelling.data_generator(images_patched, masks_patched)
# trained_model = modelling.training(param_config.get("epochs"), images_patched, masks_patched, folder_config.get("models_folder"), "test1")

Found 8519 images belonging to 1 classes.
Found 2129 images belonging to 1 classes.
Found 8519 images belonging to 1 classes.
Found 2129 images belonging to 1 classes.


## STEP 5:
    generating predictions and reverting them to the size of the original images

In [6]:
modelling.predict_folder(folder_config.get("test_folder"),folder_config.get("data_predictions"),folder_config.get("models_folder"),"best_model_root_masks")

segmentation.return_original_size(folder_config.get("data_predictions"), folder_config.get("test_folder"), folder_config.get("data_predictions_clean"))

prediction = segmentation.test_overlaying(folder_config.get("test_folder"), folder_config.get("data_predictions_clean"), 9, alpha=0.35)

INFO:root:Predicting 030_43-19-ROOT1-2023-08-08_pvdCherry_OD001_Col0_04-Fish Eye Corrected.png


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 480ms/step


INFO:root:Predicting 030_43-2-ROOT1-2023-08-08_pvdCherry_OD001_Col0_05-Fish Eye Corrected.png


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 436ms/step


INFO:root:Predicting 031_43-18-ROOT1-2023-08-08_pvd_OD0001_col-0_05-Fish Eye Corrected.png


...

INFO:root:Predicting 040_43-14-ROOT1-2023-08-08_pvd_OD001_Col0_01-Fish Eye Corrected.png


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 442ms/step


INFO:root:Predicting 040_43-18-ROOT1-2023-08-08_pvd_OD01_Col0_03-Fish Eye Corrected.png


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 435ms/step


100%|          | 0/30 [00:00<?, ?it/s]▎         | 1/30 [00:00<00:10,  2.67it/s]▋         | 2/30 [00:00<00:10,  2.67it/s]█         | 3/30 [00:01<00:10,  2.67it/s]█▎        | 4/30 [00:01<00:09,  2.68it/s]█▋        | 5/30 [00:01<00:09,  2.68it/s]██        | 6/30 [00:02<00:08,  2.69it/s]██▎       | 7/30 [00:02<00:08,  2.68it/s]██▋       | 8/30 [00:02<00:08,  2.67it/s]███       | 9/30 [00:03<00:07,  2.67it/s]███▎      | 10/30 [00:03<00:07,  2.68it/s]███▋      | 11/30 [00:04<00:07,  2.69it/s]████      | 12/30 [00:04<00:06,  2.68it/s]████▎     | 13/30 [00:04<00:06,  2.69it/s]████▋     | 14/30 [00:05<00:05,  2.68it/s]█████     | 15/30 [00:05<00:05,  2.70it/s]█████▎    | 16/30 [00:05<00:05,  2.71it/s]█████▋    | 17/30 [00:06<00:04,  2.70it/s]██████    | 18/30 [00:06<00:04,  2.68it/s]██████▎   | 19/30 [00:07<00:04,  2.69it/s]██████▋   | 20/30 [00:07<00:03,  2.70it/s]███████   | 21/30 [00:07<00:03,  2.68it/s]███████▎  | 22/30 [00:08<00:02,  2.70it/s]███████▋  | 23/30 [00:08<00:02,  2.69it/s]█████

## STEP 6:
      detecting the root tips landmarks
      NOTE: I got 26/30 from the test set correctly masks 1, 3, 5, and 29 have mislocated root coordinates.

In [9]:
image_num = 29
landmarks.display_root_landmarks(folder_config.get("data_predictions_clean"),
                                 folder_config.get("test_folder"),
                                 image_num)

y, x = landmarks.get_bottom_coordinates(folder_config.get("data_predictions_clean"),image_num)
logging.info(f"coordinates x: {x}" f" \ncoordinates y: {y}")

INFO:root:coordinates x: (987, 1731, 2134, 2744, 3050) 
coordinates y: (1692, 1692, 679, 1562, 1776)


In [10]:
image_num = 27
landmarks.display_root_landmarks(folder_config.get("data_predictions_clean"),
                                 folder_config.get("test_folder"),
                                 image_num)

y, x = landmarks.get_bottom_coordinates(folder_config.get("data_predictions_clean"),image_num)
logging.info(f"coordinates x: {x}" f" \ncoordinates y: {y}")

INFO:root:coordinates x: (1239, 1725, 2173) 
coordinates y: (1585, 1844, 714)


In [11]:
image_num = 23
landmarks.display_root_landmarks(folder_config.get("data_predictions_clean"),
                                 folder_config.get("test_folder"),
                                 image_num)

y, x = landmarks.get_bottom_coordinates(folder_config.get("data_predictions_clean"),image_num)
logging.info(f"coordinates x: {x}" f" \ncoordinates y: {y}")

INFO:root:coordinates x: (1234, 1697, 2185, 2705, 3245) 
coordinates y: (1042, 1077, 1126, 1152, 1142)


In [12]:
image_num = 25
landmarks.display_root_landmarks(folder_config.get("data_predictions_clean"),
                                 folder_config.get("test_folder"),
                                 image_num)

y, x = landmarks.get_bottom_coordinates(folder_config.get("data_predictions_clean"),image_num)
logging.info(f"coordinates x: {x}" f" \ncoordinates y: {y}")

INFO:root:coordinates x: (826, 1993, 2271, 2481, 3266) 
coordinates y: (1427, 1333, 1413, 1382, 1554)


In [13]:
image_num = 22
landmarks.display_root_landmarks(folder_config.get("data_predictions_clean"),
                                 folder_config.get("test_folder"),
                                 image_num)

y, x = landmarks.get_bottom_coordinates(folder_config.get("data_predictions_clean"),image_num)
logging.info(f"coordinates x: {x}" f" \ncoordinates y: {y}")

INFO:root:coordinates x: (1359, 1588, 2214, 2637, 3427) 
coordinates y: (1867, 1941, 1872, 1643, 1713)


## STEP 7:
    Converting from the pixel-space to mm-space

In [3]:
import pandas as pd
import numpy as np
import random as random

In [6]:
image_num = random.randint(0, 29)
y, x = landmarks.get_bottom_coordinates(folder_config.get("data_predictions_clean"),image_num)
df = create_and_sort_dataframe(y, x)

5 root tips coordinates found
      X     Y  Z
1  1653  1177  0
0  1172  1121  0
3  2702  1112  0
2  2149   989  0
4  3163   946  0


In [7]:
for i in range(len(df)):
    print(get_image_coordinates(df, i))

Scaled coordinates of root tip [468.03343023 451.10911754]
[0.16456148 0.14607347 0.057     ]
Scaled coordinates of root tip [660.11882267 473.64445258]
[0.16739952 0.16990737 0.057     ]
Scaled coordinates of root tip [858.19440407 397.99011351]
[0.15787181 0.19448454 0.057     ]
Scaled coordinates of root tip [1079.03270349  447.48736726]
[0.16410537 0.2218861  0.057     ]
Scaled coordinates of root tip [1263.13117733  380.68619553]
[0.15569261 0.24472899 0.057     ]
